In [1]:
import pandas as pd
import numpy as np 
from multiprocessing import Pool
from tqdm import tqdm
import os
import torch 
import warnings
import json 
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 500)
pd.set_option("display.precision", 1)

In [2]:
# image_folder = "/home/user/DataCenter/HAR_data/house_images/"
# img_paths = []
# for i in tqdm(range(100)):
#     for j in range(100):
#         subfolder = os.path.join(image_folder, "%04d/%04d/"%(i,j))
#         img_paths.extend([os.path.join(subfolder, f_) for f_ in os.listdir(subfolder)])

In [3]:
raw_df = pd.read_pickle("../../data/property_structured_12162024.pkl")
raw_df.head(2)

,address,harlink,mp_features,agent,price,posted_days,status,zipcode,num_beds,num_bath,num_half_bath,building_sqft,has_loft,num_stories,num_parking_space,address_key,date,json_path,time_epoch,image_path,json_size,image_cnt,LAT,LON,building_sqft2,...,Baths,Maintenance Fee,Tax Rate,tax_rate2,Recent Market Value,Recent Tax Value,Property Type,County,Private Pool,Area Pool,Private_pool_desc,Pool_feature_multiclass,Foundation_multiclass,Garage Types_multiclass,Roof Type_multiclass,floor_type_multiclass,Exterior Type_multiclass,Exterior_multiclass,Style_multiclass,finance_option_multiclass,high_school_star,middle_school_star,elementary_school_star,school_org,general_desc
57,"4513 Refugio Dr, Plano, TX 75024",https://www.har.com//homedetail/4513-refugio-d...,"4 bedrooms 3,395 Sqft. ($233/Sqft.) 3 full & 1...",Karla Davis Fathom Realty,789990.0,66.0,Under Contract - P,75024,4.0,0.0,0.0,3395.0,0.0,0.0,0.0,"4513 Refugio Dr, Plano, TX 75024",2023_11_05,/home/user/DataCenter/HAR_data/address_fullinf...,1690411643,/home/user/DataCenter/HAR_data/house_images/00...,4425,12,-96.8,33.1,3395.0,...,3.5,65.1,1.9,NaN,157533.0,157533.0,single family,Collin County,No,None,None,[],[Slab],[Attached],[composition],"[carpet, ceramic tile, wood]","[brick, rock/stone]",[covered patio/porch],[traditional],[],NaN,5.0,5.0,[FRISCO ISD],"MOVE IN READY! Welcome to The Bartlet, this ge..."
8,"1516 Bay Area Blvd P12, Houston, TX 77058",https://www.har.com//homedetail/1516-bay-area-...,1 bedrooms 684 Sqft. ($117/Sqft.) 1 full baths...,Laurie Long RE/MAX Space Center,79900.0,79.0,Under Contract - PS,77058,1.0,0.0,0.0,684.0,0.0,0.0,0.0,"1516 Bay Area Blvd P12, Houston, TX 77058",2023_11_05,/home/user/DataCenter/HAR_data/address_fullinf...,1689375459,/home/user/DataCenter/HAR_data/house_images/00...,5661,11,-95.1,29.6,684.0,...,1.0,272.0,2.4,2.4,69024.0,47052.0,townhouse/condo,Harris County,No,Yes,None,[],[Slab],[],[composition],"[tile, vinyl]","[brick, stucco]","[balcony, partially fenced, storage]",[traditional],"[cash sale, conventional, fha, investor, va]",5.0,NaN,4.0,[CLEAR CREEK ISD],This is the perfect starter home! EVERYTHING U...


In [4]:
## Get the list of images to be encoded
def get_image_paths(image_folder, check_mean=True):
    images_path = [os.path.join(image_folder, f_) for f_ in os.listdir(image_folder) if f_.endswith(".jpeg")]
    n_images = len(images_path)
    if check_mean:
        images_path = [f_ for f_ in images_path if os.path.exists(f_.replace(".jpeg", "mean.pt")) is False] 
    else:
        images_path = [f_ for f_ in images_path if os.path.exists(f_.replace(".jpeg", ".pt")) is False]
    return images_path, n_images 

batch_size = 2048 
n_batches = len(raw_df) // batch_size + 1
all_img_cnt = 0
all_img_paths = []

for batch_i in tqdm(range(n_batches)):
    with Pool(10) as p:
        img_path, img_cnt = zip(*p.map(get_image_paths, raw_df["image_path"].iloc[batch_i * batch_size: (batch_i + 1) * batch_size].values))
    [all_img_paths.extend(path_) for path_ in img_path]
    all_img_cnt += sum(img_cnt)


00%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 104/104 [00:40<00:00,  2.55it/s]

In [5]:
"There are %i images, and %i to be encoded"%(all_img_cnt, len(all_img_paths))

'There are 4239179 images, and 1006923 to be encoded'

## Run Dino Encoding 

In [6]:
## Run DINO Encoding
import torch.utils.data as data_utils
from torchvision import datasets, transforms
import torch
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import requests

device = "cuda:0"
processor = AutoImageProcessor.from_pretrained('facebook/dinov2-small')
model = AutoModel.from_pretrained('facebook/dinov2-small')
model.cuda(device)

def read_image(image_path):
    image = Image.open(image_path)
    return image

preprocessor_config.json:   0%|          | 0.00/436 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/88.2M [00:00<?, ?B/s]

In [7]:
batch_size = 1024 
n_batches = (len(all_img_paths) + batch_size - 1) // batch_size

for batch_i in tqdm(range(n_batches)):
    batch_images = all_img_paths[batch_i * batch_size: batch_i * batch_size + batch_size]
    with Pool(10) as p:
        inp_imgs = p.map(read_image, batch_images)
    inputs = processor(images=inp_imgs, return_tensors="pt").to(device, non_blocking=True)
    with torch.no_grad():
        last_hidden_states = model(**inputs)[0]
        embedding_list = last_hidden_states[:, 0, :].cpu()
        mean_embedding_list = torch.mean(last_hidden_states, axis=1).cpu()
    embedding_list = [e.clone() for e in embedding_list]
    mean_embedding_list = [e.clone() for e in mean_embedding_list]
    # _ = [torch.save(embedding_list[i], batch_images[i].replace(".jpeg", ".pt")) for i in range(len(embedding_list))]
    _ = [torch.save(mean_embedding_list[i], batch_images[i].replace(".jpeg", "mean.pt")) for i in range(len(embedding_list))]


00%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 984/984 [2:50:37<00:00, 10.40s/it]

In [83]:
# dir(embedding_list[0])

### Stack the image embeddings 

In [5]:
import pandas as pd 
import numpy as np 
import torch 
import os 
import warnings
import json 
from multiprocessing import Pool
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 500)
pd.set_option("display.precision", 1)
raw_df = pd.read_pickle("../../data/property_structured_12162024.pkl")

In [ ]:
def load_and_stack_tensor(image_folder, use_mean_tensor=False):
    if use_mean_tensor:
        pt_paths = [os.path.join(image_folder, f_) for f_ in os.listdir(image_folder) if f_.endswith("mean.pt")]
    else: 
        pt_paths = [os.path.join(image_folder, f_) for f_ in os.listdir(image_folder) if f_.endswith(".pt") and "mean" not in f_]
    if len(pt_paths) == 0:
        return np.zeros(384)
    pt_list = np.array([torch.load(pt_path).numpy() for pt_path in pt_paths])
    return np.mean(pt_list, axis=0)

with Pool(10) as p:
    image_dinov2s_cls_mean = p.map(load_and_stack_tensor, raw_df["image_path"].values)

In [6]:
def load_and_stack_tensor(image_folder, use_mean_tensor=True):
    if use_mean_tensor:
        pt_paths = [os.path.join(image_folder, f_) for f_ in os.listdir(image_folder) if f_.endswith("mean.pt")]
    else: 
        pt_paths = [os.path.join(image_folder, f_) for f_ in os.listdir(image_folder) if f_.endswith(".pt") and "mean" not in f_]
    if len(pt_paths) == 0:
        return np.zeros(384)
    pt_list = np.array([torch.load(pt_path).numpy() for pt_path in pt_paths])
    return np.mean(pt_list, axis=0)
    
with Pool(10) as p:
    image_dinov2s_mean_mean = p.map(load_and_stack_tensor, raw_df["image_path"].values)

In [8]:
# raw_df["img_emb"] = image_dinov2s_mean_mean
# raw_df[["address", "address_key", "img_emb"]].to_pickle("../../data/property_structured_12162024_image_dinov2s_mean_mean.pkl")